In [2]:
import re
import pandas as pd
import itertools
import cobra.flux_analysis
import time
from functools import partial
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Recon3D/JSON/Recon3D.json'
model = cobra.io.load_json_model(model_file_path)


SNP_results = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_non_synonymous.tsv', index_col=0)

end_time = time.time()
print('Model load time: %.6f seconds' % (end_time - start_time))

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Model load time: 18.739216 seconds


In [4]:
# Prepare SNP combinations; lists genes to knock out by model id.
combinations = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNP_combinations.tsv', index_col=0)
combinations['combinations'] = combinations['combinations'].apply(lambda x: x.split(';'))

combinations['gene_model_ids'] = combinations['combinations']\
    .apply(lambda x: SNP_results.loc[SNP_results['variant_name'].isin(x), ['model_gene_number']].iloc[:, 0].tolist())

id_list = ';' + ';'.join(model.genes.list_attr('id'))

combinations['gene_model_ids'] = combinations['gene_model_ids'].apply(lambda x:
                                list(set(itertools.chain.from_iterable(
                                [re.findall(r"(?:;)(" + str(i) + r"_AT\d+)", id_list) for i in x]))))

combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test.tsv', sep='\t')

In [5]:
# FBA code cell
start_time = time.time()


combinations = parallelize_dataframe(combinations, partial(combinations_subset, partial(knockout_FBA, model)), 8)
combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test.tsv', sep='\t')


end_time = time.time()
print('FBA run time: %.6f seconds' % (end_time - start_time))

TypeError: cannot pickle 'SwigPyObject' object

In [9]:
# Check for non nominal results
for i, result in combinations.iterrows():
    res = result['results'].objective_value
    if int(res) < 700:
        print(result)



combinations      [rs144738806, rs58212531, rs114264176, rs56677...
gene_model_ids    [170384_AT1, 8802_AT1, 18_AT2, 478_AT1, 1244_A...
results                         <Solution 674.472 at 0x1dbe9f57d30>
Name: 80, dtype: object
combinations      [rs536105762, rs57058990, rs182401722, rs15052...
gene_model_ids    [50484_AT1, 411_AT1, 3098_AT1, 4645_AT1, 79581...
results                         <Solution 699.461 at 0x1dbe609c670>
Name: 111, dtype: object
combinations      [rs565524272, rs114264176, rs550568353, rs1405...
gene_model_ids    [8802_AT1, 27430_AT1, 10846_AT1, 80201_AT1, 94...
results                         <Solution 674.472 at 0x1dbe624b310>
Name: 162, dtype: object
combinations      [rs568979468, rs545762746, rs542133702, rs5316...
gene_model_ids    [491_AT1, 245973_AT1, 8802_AT1, 55315_AT1, 108...
results                         <Solution 674.472 at 0x1dbe622e250>
Name: 280, dtype: object
combinations      [rs546991277, rs114264176, rs565731657, rs5580...
gene_model_ids   

In [23]:
def get_model_gene_info():
    """Function to extract all unique ids, not including "_ATXX" to a file"""
    ids = model.genes.list_attr('id')
    gene_ids = []

    for i in ids[1:]:
        s = i.split('_')
        gene_ids.append(int(s[0]))


    results = pd.Series(gene_ids, name='gene_number').drop_duplicates().sort_values().reset_index(drop=True)
    results.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/exon_model_data/gene_ids_in_model.tsv', sep='\t')


get_model_gene_info()

In [34]:
# It is somewhat convoluted, but not sure how to do it differently.
def get_recon_store_names_by_model_id():
    """function to get gene names by using ids from the model.
    Needed because the names in the model and from the store/ensembl are not necessarily the same.
    The ensembl_gene ids are not necessarily the same due to alternative sequence genes.
    Names are not always the same in the model itself and in the vmh website."""

    gene_ids = pd.read_table('C:/Users/Sigve/Genome_Data/exon_model_data/gene_ids_in_model.tsv', index_col=0)
    recon_store = pd.read_table('C:/Users/Sigve/Genome_Data/exon_model_data/recon-store-genes.tsv')
    recon_store['gene_number'] = recon_store['gene_number'].astype(int)

    new_df = pd.merge(gene_ids, recon_store, on='gene_number')
    new_df = new_df[['gene_number', 'symbol', 'chromosome', 'ensembl_gene']]
    new_df.drop_duplicates(subset=['gene_number'], inplace=True)
    new_df.rename(columns={'symbol': 'gene_name'}, inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    new_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/exon_model_data/model_gene_ids.tsv', sep='\t')
    print(new_df.head())

get_recon_store_names_by_model_id()

   gene_number gene_name chromosome     ensembl_gene
0           13     AADAC          3  ENSG00000114771
1           15     AANAT         17  ENSG00000129673
2           18      ABAT         16  ENSG00000183044
3           19     ABCA1          9  ENSG00000165029
4           21     ABCA3         16  ENSG00000167972


In [7]:
# Test stuff

#print(model.reactions.HMR_9505.gene_name_reaction_rule)
#model.reactions.MAOX.genes


gene = model.genes.get_by_id('8644_AT1')
print(gene.__getattribute__('name'))
gene

Gene identifier,ENSG00000131069
Name,
Memory address,0x029cbd3f19d0
Functional,True
In 10 reaction(s),"MAR04459, MAR03796, MAR11394, MAR04108, MAR04097, MAR04133, MAR04460, MAR03797, MAR00153, MAR04331"


In [20]:
# Code to change format of SNP files to be compatible with other code.
new_SNPs = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/SNPs_all_chrom_orig.tsv')
columns = ['rsids', 'Variant source', 'chrom', 'pos', 'ref', 'alt']
new_SNPs = new_SNPs.reindex(columns=columns)
new_SNPs.rename(columns={'rsids': 'Variant name', 'chrom': 'Chromosome/scaffold name', 'pos': 'Chromosome/scaffold position start (bp)'}, inplace=True)
new_SNPs['Chromosome/scaffold position end (bp)'] = new_SNPs['Chromosome/scaffold position start (bp)']
new_SNPs['Variant alleles'] = new_SNPs['ref'] + '/' + new_SNPs['alt']
new_SNPs.drop(columns=['ref', 'alt'], inplace=True)
new_SNPs['Strand'] = 1
new_SNPs['Chromosome/scaffold position start (bp)'] = new_SNPs['Chromosome/scaffold position start (bp)'].apply(int)
new_SNPs['Chromosome/scaffold position end (bp)'] = new_SNPs['Chromosome/scaffold position end (bp)'].apply(int)
new_SNPs.set_index(['Variant name'], drop=True, inplace=True)
print(new_SNPs.head())

new_SNPs.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/SNP_data/SNPs_all_chrom.tsv', sep='\t')

              Variant source  Chromosome/scaffold name  \
Variant name                                             
rs1000036                NaN                        14   
rs1000036                NaN                        14   
rs10000745               NaN                         4   
rs1000108                NaN                         8   
rs10001104               NaN                         4   

              Chromosome/scaffold position start (bp)  \
Variant name                                            
rs1000036                                    22772136   
rs1000036                                    22772136   
rs10000745                                  126429447   
rs1000108                                    27429649   
rs10001104                                    1527546   

              Chromosome/scaffold position end (bp) Variant alleles  Strand  
Variant name                                                                 
rs1000036                            